In [1]:
import tensorflow as tf

2021-12-10 02:38:53.182743: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-12-10 02:38:53.182771: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
tf.__version__

'2.7.0'

In [3]:
import os
import numpy as np
import cv2
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt
from math import ceil

In [4]:
from chessutils import find_coeffs

# Datagenerator

In [5]:
PATH_TO_IMG = 'img'

In [6]:
CELL = 45
NUMCELL = 8
BOARDSIZE = CELL*NUMCELL

In [7]:
IMGSIZE = 480
MAXSHEAR = 0.15
SIZE = IMGSIZE // NUMCELL

In [8]:
figimgs = [f for f in os.listdir(PATH_TO_IMG) if f.split('_')[0]=='Chess']
figuresimgs = dict()
for f in figimgs:
    fn = f.split('_')[1].split('4')[0]
    img = cv2.imread(os.path.join(PATH_TO_IMG, f))
    figuresimgs[fn] = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

figs = ['x', 'p', 'b', 'n', 'r', 'q', 'k']
codelist = np.array(['xx', 'pd', 'pl', 'bd', 'bl', 'nd', 'nl', 'rd', 'rl', 'qd', 'ql', 'kd', 'kl'])

In [9]:
def distort(img, distortrnd):
    ns = img.size[0]
    m = (distortrnd[:8]) * 0.25 * ns

    x1, y1 = -m[0:2]
    x2, y2 = ns+m[2], m[3]

    x3, y3 = ns+m[4], ns-m[5]
    x4, y4 = -m[6], ns+m[7]

    xmin = min(x1, x4)
    if (xmin < 0):
        x1 -= xmin
        x2 -= xmin
        x3 -= xmin
        x4 -= xmin

    ymin = min(y1, y2)
    if (ymin < 0):
        y1 -= ymin
        y2 -= ymin
        y3 -= ymin
        y4 -= ymin

    new_width = int(ceil(max(x2,x3,ns)))
    new_height = int(ceil(max(y3,y4,ns)))

    coeffs = find_coeffs(
        [(x1, y1), (x2, y2), (x3, y3), (x4, y4)],
        [(0, 0), (ns, 0), (ns, ns), (0, ns)])
    img = img.transform((new_width, new_height), Image.PERSPECTIVE, coeffs, Image.BICUBIC, fillcolor = 0)
    return img


In [10]:
def moire(rndarr, sz):
    '''
    distortrnd(8)
    '''
    ns = 2*sz
    grey = 64
    blank = (255-grey) + grey*np.ones((ns,ns), np.uint8)
    for i in range(ns):
        if i % 3 != 0:
            continue
        blank[i] = grey*np.ones(ns, np.uint8)
    img = Image.fromarray(blank)

    rndxxx = rndarr*np.array([1,-1,1,-1,1,-1,1,-1])
    
    im1 = distort(img, rndarr)
    im2 = distort(img, rndxxx)
    
    xs = max(im1.size[0], im2.size[0])
    ys = max(im1.size[1], im2.size[1])
    
    im1 = im1.resize((xs,ys)) 
    im2 = im2.resize((xs,ys)) 
    
    re1 = np.asarray(im1)
    re2 = np.asarray(im2)
    
    ims = Image.fromarray(np.maximum(im1,im2))
    ims = ims.crop((sz//2, sz//2, sz//2+sz, sz//2+sz))
    ims = ims.resize((120,120))
    ims = ims.resize((sz,sz))
    
    res = np.asarray(ims) / 255
    
    maxvalshift = 1 - res.max()
    res += maxvalshift

    return res

In [11]:
def boarddistortgen():
    blank_image = np.zeros((BOARDSIZE,BOARDSIZE), np.uint8)
    codedvec = np.random.randint(7,size=NUMCELL*NUMCELL)
    codedcol = np.zeros(NUMCELL*NUMCELL)
    for i, cod in enumerate(codedvec):
        xp = i % NUMCELL
        yp = i // NUMCELL
        dice = np.random.randint(2)
        figcolcod = 0
        codedcol[i] = 0
        if cod != 0:
            codedcol[i] = dice+1
            figcolcod = cod*2-dice
        fldcol = 'l' if (xp+yp) % 2 == 0 else 'd'
        fkey = f'{codelist[figcolcod]}{fldcol}'
        blank_image[yp*CELL:(yp+1)*CELL,xp*CELL:(xp+1)*CELL] = figuresimgs[fkey]

    img = Image.fromarray(blank_image, 'L')
    distortrnd = np.random.rand(13)

    m = (2 * distortrnd[:8] - 1) * MAXSHEAR * BOARDSIZE

    x1, y1 = m[0:2]
    x2, y2 = BOARDSIZE+m[2], m[3]
    x3, y3 = BOARDSIZE+m[4], BOARDSIZE+m[5]
    x4, y4 = m[6], BOARDSIZE+m[7]

    xmin = min(x1, x4)
    if (xmin < 0):
        x1 -= xmin
        x2 -= xmin
        x3 -= xmin
        x4 -= xmin

    ymin = min(y1, y2)
    if (ymin < 0):
        y1 -= ymin
        y2 -= ymin
        y3 -= ymin
        y4 -= ymin

    greyscale = int((0.5 + distortrnd[8]*0.5) * 255)

    new_width = int(ceil(max(x2,x3,BOARDSIZE)))
    new_height = int(ceil(max(y3,y4,BOARDSIZE)))

    coeffs = find_coeffs(
        [(x1, y1), (x2, y2), (x3, y3), (x4, y4)],
        [(0, 0), (BOARDSIZE, 0), (BOARDSIZE, BOARDSIZE), (0, BOARDSIZE)])
    
    img = img.transform((new_width, new_height), Image.PERSPECTIVE, coeffs, Image.BICUBIC, fillcolor = greyscale)

    offsets = distortrnd[9:11]
    width, height = img.size

    xoff = (IMGSIZE - width) * offsets[0]
    yoff = (IMGSIZE - height) * offsets[1]

    offset = (int(round(xoff)), int(round(yoff)))

    background = moire(np.random.rand(8), IMGSIZE)

    empty = Image.new('L', (IMGSIZE, IMGSIZE), greyscale)
    empty.paste(img, offset)

    result = (background * np.asarray(empty)).astype(np.uint8)

    vec = np.array([x1+offset[0], y1+offset[1], x2+offset[0], y2+offset[1], x3+offset[0], y3+offset[1], x4+offset[0], y4+offset[1]])
    distortedvec = vec + 20 * (np.random.rand(8) - 0.5)
    distortedvec = np.clip(distortedvec, 0, IMGSIZE-1)
    
    coeffs = find_coeffs(
        [(0, 0), (IMGSIZE, 0), (IMGSIZE, IMGSIZE), (0, IMGSIZE)],
        np.reshape(distortedvec, (4,2)))
    
    img = Image.fromarray(result, 'L')
    img = img.transform((IMGSIZE,IMGSIZE), Image.PERSPECTIVE, coeffs, Image.BICUBIC, fillcolor = greyscale)

    brd = np.asarray(img) / 255

    nrand = np.random.randint(NUMCELL*NUMCELL)
    nrand = 0
    xrand = nrand % NUMCELL
    yrand = nrand // NUMCELL
    
    coded = np.array([codedvec[nrand], codedcol[nrand]])
    
    brd_out = np.expand_dims(brd[yrand*SIZE:(yrand+1)*SIZE,xrand*SIZE:(xrand+1)*SIZE], axis=-1)
    
    yield brd_out, coded

In [12]:
dataset = tf.data.Dataset.from_generator(boarddistortgen,
                                         output_signature=
                                         (tf.TensorSpec(shape=(SIZE,SIZE,1), dtype=tf.float32),
                                          tf.TensorSpec(shape=(2), dtype=tf.float32))
                                        )

2021-12-10 02:39:00.515604: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-12-10 02:39:00.515637: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2021-12-10 02:39:00.515663: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ilia-ThinkPad-S5-S540): /proc/driver/nvidia/version does not exist
2021-12-10 02:39:00.515926: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
trainset = dataset.repeat().batch(64)
valset = dataset.repeat(150).batch(64)

# CLASSIFIER

In [14]:
from keras.models import Model
from keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dense, MaxPooling2D, Concatenate

In [15]:
def classnet():

    inputs = tf.keras.layers.Input(shape=(SIZE,SIZE,1))

    x = inputs
    fsize = [16, 32, 64, 128]

    for i in range(4):
        x = Conv2D(filters = fsize[i],
                kernel_size = (3, 3),
                kernel_initializer = 'he_normal',
                padding = 'same', strides=(2,2), activation='relu', name=f'cnv1_{i}')(x)

    x = GlobalAveragePooling2D(name='glob')(x)
    output_fig = Dense(7, activation = 'softmax', name='fig')(x)
    output_col = Dense(3, activation = 'softmax', name='col')(x)
    output = Concatenate(name='unite')([output_fig, output_col])
    
    model = Model(inputs=inputs, outputs=output, name='class_board')

    return model

In [16]:
md = classnet()

In [17]:
md.summary()

Model: "class_board"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 60, 60, 1)]  0           []                               
                                                                                                  
 cnv1_0 (Conv2D)                (None, 30, 30, 16)   160         ['input_1[0][0]']                
                                                                                                  
 cnv1_1 (Conv2D)                (None, 15, 15, 32)   4640        ['cnv1_0[0][0]']                 
                                                                                                  
 cnv1_2 (Conv2D)                (None, 8, 8, 64)     18496       ['cnv1_1[0][0]']                 
                                                                                        

In [ ]:
def custom_loss(y_actual, y_pred):

    scc = tf.keras.losses.SparseCategoricalCrossentropy()
    part1 = scc(y_actual[:,0:1],y_pred[:,0:7])
    part2 = scc(y_actual[:,1:2],y_pred[:,7:10])

    return part1 + part2

In [ ]:
class colAccuracy(tf.keras.metrics.SparseCategoricalAccuracy):

    def __init__(self, name='col_acc', **kwargs):
        super(colAccuracy, self).__init__(name=name, **kwargs)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_actual = y_true[:,1:2]
        y_predicted = y_pred[:,7:10]
        return super(colAccuracy,self).update_state(y_actual,y_predicted,sample_weight)

In [ ]:
class figAccuracy(tf.keras.metrics.SparseCategoricalAccuracy):

    def __init__(self, name='fig_acc', **kwargs):
        super(figAccuracy, self).__init__(name=name, **kwargs)

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_actual = y_true[:,0:1]
        y_predicted = y_pred[:,0:7]
        return super(figAccuracy,self).update_state(y_actual,y_predicted,sample_weight)

In [ ]:
md.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=custom_loss,
               metrics=[figAccuracy(), colAccuracy()])

In [ ]:
md.evaluate(valset)

In [ ]:
EPOCHS = 10
model_history = md.fit(trainset, steps_per_epoch=50, epochs=EPOCHS, validation_data=valset)

In [ ]:
md.save('models/class_figure_col_v1.h5')

In [ ]:
##################################################################